In [1]:
!pip install datasets
!pip install -U sentence-transformers transformers
from datasets import Dataset
import os
import datasets
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses, util
from torch.utils.data import DataLoader
from nltk.stem import WordNetLemmatizer
import nltk
import gc
import torch
os.environ["WANDB_MODE"] = "disabled"
nltk.download("wordnet")

data = pd.read_csv('merged_unlabeled_all.csv', encoding='utf-8', on_bad_lines='skip')

lemmatizer = WordNetLemmatizer()
def lemmatize_lyrics(text):
    return ' '.join(['ass' if word == 'ass' else lemmatizer.lemmatize(word) for word in text.split()])

data['lyrics'] = data['lyrics'].apply(lambda x: lemmatize_lyrics(str(x)))

category = 'Substance'
keyword_col = 'Substance_words'
keywords = data[keyword_col].dropna().tolist()
split_keywords = []
for phrase in keywords:
    split_keywords.extend([kw.strip() for kw in phrase.split(',')])
all_keywords_cleaned = list(set(split_keywords))

print(f"\n{category} Keywords ({len(all_keywords_cleaned)} total):")
print(", ".join(all_keywords_cleaned))

model = SentenceTransformer('all-MiniLM-L6-v2')

keyword_embeddings = {keyword: model.encode(keyword, convert_to_tensor=True) for keyword in all_keywords_cleaned}

threshold = 0.62
min_threshold = 0.56
window_size = 12

train_examples = []
for index, row in data.iterrows():
    lyric = row['lyrics']
    label = row[category]
    if label == 'T':
        for keyword in all_keywords_cleaned:
            if keyword in lyric:
                start = max(0, lyric.find(keyword) - window_size)
                end = min(len(lyric.split()), start + window_size * 2 + 1)
                segment = ' '.join(lyric.split()[start:end])
                train_examples.append(InputExample(texts=[segment, segment], label=1.0))

bootstrapping_active = True
round = 1
while bootstrapping_active:
    print("=====================================================================================================")
    print("**Current threshold: ", threshold, " | Current Round: ", round)
    model = SentenceTransformer('all-MiniLM-L6-v2')

    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
    train_loss = losses.CosineSimilarityLoss(model)

    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=3,
        warmup_steps=100,
        show_progress_bar=False
    )

    for index, row in data.iterrows():
        if pd.isna(row[category]):
            lyric = row['lyrics']
            words = lyric.split()
            is_true = False

            for i in range(0, len(words) - 25 + 1, 6):
                segment = ' '.join(words[i:i + 25])
                segment_embedding = model.encode(segment, convert_to_tensor=True)

                similarity_score = max(
                    util.pytorch_cos_sim(segment_embedding, keyword_embedding).item()
                    for keyword_embedding in keyword_embeddings.values()
                )

                if similarity_score > threshold:
                    data.at[index, category] = 'M'
                    print("Threshold: ", threshold, " | Similarity Score: ", similarity_score)
                    print(f"New 'T' sample in {category} category: Row {index + 1}, Segment: '{segment}'")

                    train_examples.append(InputExample(texts=[segment, segment], label=1.0))
                    break

            del segment_embedding
            torch.cuda.empty_cache()

    if threshold > min_threshold:
        threshold -= 0.02
    else:
        bootstrapping_active = False

    # del train_dataloader, train_loss, model
    # torch.cuda.empty_cache()
    # gc.collect()

# data[category].fillna('F', inplace=True)

output_path = 'processed_lyrics_with_labels_Substance.csv'
data.to_csv(output_path, encoding='utf-8', index=False)
print(f"Final output saved.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: 

[nltk_data] Downloading package wordnet to /root/nltk_data...



Substance Keywords (20 total):
vodka, booze, alcohol, drug, pill, heroin, lager, liquor, wine, ecstasy, cocaine, intoxicated, weed, marijuana, beer, cigar, meth, whiskey, ganja, cannabis


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**Current threshold:  0.62  | Current Round:  1


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


{'train_runtime': 5.4444, 'train_samples_per_second': 158.144, 'train_steps_per_second': 9.918, 'train_loss': 0.029898460264559144, 'epoch': 3.0}
Threshold:  0.62  | Similarity Score:  0.6250781416893005
New 'T' sample in Substance category: Row 61, Segment: 'feel hot yeah yeah spill the wine spill the wine spill the wine spill the wine spill the wine spill the wine spill the wine'
Threshold:  0.62  | Similarity Score:  0.6243101358413696
New 'T' sample in Substance category: Row 468, Segment: 'with this handsome kid ciga cigar right from cuba cuba i just bite it it for the look i dont light it illway the an'
Threshold:  0.62  | Similarity Score:  0.621188759803772
New 'T' sample in Substance category: Row 1101, Segment: 'no holding back another shot of vodka you know whats in my glass it my house just relax welcome to my house baby take control'
**Current threshold:  0.6  | Current Round:  1
{'train_runtime': 2.5498, 'train_samples_per_second': 341.198, 'train_steps_per_second': 22.35